In [1]:
import gurobipy as gp
import pandas as pd
import geopandas as gpd
from gurobipy import Model, GRB, quicksum
import numpy as np
import pickle
import os

In [2]:
from typing import Dict, Any

In [3]:
# Códigos IBGE dos distritos (e bairros, onde aplicável) de Sergipe
origem_alvo_df = pd.read_csv('Downloads/SE_coords_medias_divisoes.csv')

# Códigos IBGE dos estabelecimentos (hospitais gerais)
facilidades_df = pd.read_csv('Downloads/Hospitais_gerais_CNES.csv')

$t_0$              Tempo máximo aceitável de viagem

In [4]:
# Parâmetro de tempo
t_0 = 45 # tempo máximo aceitável de viagem

$t_{ij}$           Tempo de viagem do nó $i$ até a facilidade $j$

In [5]:
# Resultado do OPEN ROUTE SERVICE (distâncias entre facilidades existentes e pontos de demanda)
A_dem_fac = pd.read_csv('Documents/matriz_rotas.csv')
A_dem_dem = pd.read_csv('matriz_282x282_distancias.csv')

durations_min_fac = A_dem_fac.iloc[:, 1:].to_numpy()
durations_min_fac.shape[1]

durations_min_dem = A_dem_dem.iloc[:, 1:].to_numpy()
durations_min_dem.shape[1]

282

In [6]:
durations_min = np.hstack([durations_min_dem, durations_min_fac])
# Resultado: matriz 282 × (282 + 36) = 282 × 318

In [7]:
# IDs reais
ids_demanda = origem_alvo_df['ID_divisao'].tolist() # Divisões (distritos + bairros) - 282 itens
ids_fac = facilidades_df['CO_UNIDADE'].tolist()    # Facilidades existentes - 36 itens

# Criar nomes únicos
col_dem = [f"D_{id_}" for id_ in ids_demanda]
col_fac = [f"F_{id_}" for id_ in ids_fac]

colunas = col_dem + col_fac

# Transformar matriz final em DataFrame
A_total_df = pd.DataFrame(durations_min, columns=colunas)

# Também nomear as linhas pelos IDs de demanda
A_total_df.index = col_dem

$\mathcal{N}$      Conjunto de nós de demanda/população \\
$\mathcal{M}^e$    Conjunto de localizações de facilidades existentes \\
$\mathcal{M}^n$    Conjunto de possíveis localizações de novas facilidades \\
$\mathcal{M}$      Conjunto de todas as localizações de facilidades, isto é, $\mathcal{M} = \mathcal{M}^e \cup \mathcal{M}^n$ \\

In [8]:
# Conjuntos iniciais
N = col_dem # Divisões (distritos + bairros)
M_e = col_fac # Facilidades existentes
M_n = N # Pontos candidatos a novas facilidades
M = M_e + M_n # Conjunto total de facilidades: concatenação das listas

In [9]:
print(N)
print(M)

['D_280190005_.', 'D_280480517_2804805002', 'D_280530705_.', 'D_280430005_.', 'D_280480505_.', 'D_280067005_2800670001', 'D_280067005_2800670003', 'D_280067005_2800670002', 'D_280067005_2800670005', 'D_280067005_2800670004', 'D_280067005_2800670007', 'D_280340105_.', 'D_280067005_2800670006', 'D_280330205_.', 'D_280240315_.', 'D_280740210_.', 'D_280360905_.', 'D_280480505_2804805003', 'D_280480518_.', 'D_280590105_.', 'D_280480518_2804805005', 'D_280480518_2804805004', 'D_280480519_2804805514', 'D_280480518_2804805007', 'D_280480518_2804805006', 'D_280480518_2804805008', 'D_280290805_2802908009', 'D_280290805_2802908008', 'D_280480512_.', 'D_280660205_.', 'D_280290805_2802908010', 'D_280290805_2802908012', 'D_280480506_.', 'D_280290805_2802908011', 'D_280290805_2802908014', 'D_280290805_2802908013', 'D_280480519_2804805506', 'D_280290805_2802908015', 'D_280480519_2804805509', 'D_280140510_.', 'D_280445805_.', 'D_280730305_.', 'D_280450805_2804508010', 'D_280450805_2804508012', 'D_28067

$\mathcal{M}_i$    Conjunto de facilidades ao alcance do nó de demanda $i$, \\
                   $\mathcal{M}_i = \{ j \in \mathcal{M} : t_{ij} \le t_0 \}$ \\
$\mathcal{N}_j$    Conjunto de nós de demanda ao alcance da facilidade $j$, \\
                $\mathcal{N}_j = \{ i \in \mathcal{N} : t_{ij} \le t_0 \}$ \\

In [10]:
# Conjuntos de área de abrangência
M_i = {
    col_dem[i]: [
        colunas[j] for j in range(durations_min.shape[1]) 
        if durations_min[i, j] <= t_0]
    for i in range(durations_min.shape[0])
} # facilidades dentro da área de demanda

N_j = {
    colunas[j]: [
        col_dem[i] for i in range(durations_min.shape[0])
        if durations_min[i, j] <= t_0
    ]
    for j in range(durations_min.shape[1])
} # demandas cobertas pela facilidade

$w_{ij}$           Valor de decaimento entre $i$ e $j$

In [11]:
# Função de decaimento (do nó i a facilidade j)
w_ij = np.exp(-0.09 * durations_min)

w_df = pd.DataFrame(
    w_ij,
    index=col_dem,      # IDs das demandas
    columns=colunas     # IDs das facilidades
)

$\underline{k},\overline{k}$ Limites inferior e superior da capacidade das novas facilidades

In [12]:
# Parâmetros de capacidade
k_min = 50     # Mínimo de capacidade de leitos por hospital aberto
k_max = 150    # Máximo de capacidade de leitos por hospital aberto

$\hat{x}_j$        Capacidade da facilidade existente $j \in \mathcal{M}^e$

In [13]:
# Suprimentos existentes
x_hat = {row['CO_UNIDADE']: row['leitos'] 
         for _, row in facilidades_df.iterrows()}
x_hat = {f"F_{k}": v for k, v in x_hat.items()}
print(x_hat)

{'F_2800300002232': 166, 'F_2800300002275': 113, 'F_2800300002283': 263, 'F_2800300002372': 15, 'F_2800300002534': 107, 'F_2800300002585': 159, 'F_2805402420120': 13, 'F_2800202421488': 22, 'F_2803502421518': 42, 'F_2804502421542': 46, 'F_2802102423529': 54, 'F_2800302444259': 59, 'F_2802902477661': 86, 'F_2805902477947': 50, 'F_2803502503824': 30, 'F_2806702545829': 41, 'F_2802902546027': 15, 'F_2803002546124': 24, 'F_2800302658496': 117, 'F_2801202658542': 10, 'F_2801302745259': 38, 'F_2800302816210': 603, 'F_2800303225798': 38, 'F_2800303277755': 16, 'F_2805703559629': 47, 'F_2800303841375': 23, 'F_2800304966279': 112, 'F_2804805129753': 26, 'F_2800306003494': 203, 'F_2800306137040': 16, 'F_2801306490581': 30, 'F_2803506568343': 131, 'F_2807106695604': 19, 'F_2805606697712': 18, 'F_2802106901743': 105, 'F_2800309778969': 19}


In [14]:
## Dicionário de Demanda (d_i)

# 1. Definição dos Parâmetros (Baseado no Quadro 45 e Equação 1 do Caderno 1) [cite: 557, 632]
# Vamos criar um coeficiente médio composto para leitos gerais (Clínico + Cirúrgico)
# Considerando uma média ponderada típica:
# TAXA_INTERNACAO_GERAL = 0.055  # ~55 internações/1000 hab/ano (Conservador)
# TEMPO_MEDIO_PERMANENCIA = 5.5  # ~5.5 dias de média geral
# TAXA_OCUPACAO_ALVO = 0.80      # 80% de ocupação (Quadro 43 recomenda 75-85% para eficiência) [cite: 622]

$d_i$              Demanda da população no nó $i$

In [15]:
# Dataset com dados de população (ajustada para 2025 e proporção de adesão à planos de saúde) de 15-59 e 60+ anos
demanda_df = pd.read_csv('setores_populacao_ajustada.csv')
demanda_df = demanda_df.rename(columns={"CD_setor": "CD_SETOR"})

# Códigos IBGE dos distritos (e bairros, onde aplicável) de Sergipe
setores_df = pd.read_csv('Downloads/SE_setores_populacao.csv')

In [16]:
# Junção dos dois datasets, pelo código do setor
df_join = demanda_df.merge(setores_df, on="CD_SETOR", how="right")
# Preenchimento dos NaN com zero
df_join = df_join.fillna(0)
# Somando os leitos necessários por ID_divisão
agrupado = df_join.groupby("ID_divisao")["necessidade_total_leitos"].sum()

# Criação do Dicionário de Demanda
d_i = agrupado.to_dict()
d_i = {f"D_{k}": v for k, v in d_i.items()}

In [17]:
# Demanda ponderada - Denominador do R_j: Soma de d_i * w_ij para todos i que j cobre
DEMANDA_PONDERADA = {}

for j_label in w_df.columns:    # j_label = "F_32", "F_41", etc.
    
    demanda_total = 0.0
    
    for i_label in N_j[j_label]:    # i_label = "D_105", "D_204", etc.
        
        w = w_df.loc[i_label, j_label]
        d = d_i.get(i_label, 0.0)
        
        demanda_total += w * d
    
    DEMANDA_PONDERADA[j_label] = demanda_total

$\alpha$           Parâmetro de trade-off entre equidade e eficiência \\
$P$                Número total de facilidades a serem abertas \\
$Q$                Quantidade total de recursos/atendimentos disponíveis

In [18]:
# Definição de Limites Orçamentários e Capacidade (a serem ajustados dinamicamente)
P_max = 3           # Número exato de NOVOS centros que o governo pode abrir/financiar.
Q_max = 200         # Orçamento total de leitos NOVOS a serem contratados na rede.

# Parâmetro de Trade-off (Alpha no artigo)
# Alpha = 1: Foco total na Eficiência (Acesso Médio). Alpha = 0: Foco total na Equidade (Pior Cenário).
alpha = 0.0001

In [19]:
# Modelo (Gurobi)
m = gp.Model("Localizacao_Centros_Medicos")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2732058
Academic license 2732058 - for non-commercial use only - registered to na___@usp.br


Variáveis de decisão<br>
$y_j$              $= 1$ se uma facilidade é aberta em $j \in \mathcal{M}^n$, caso contrário $0$<br>
$x_j$              Capacidade atribuída à facilidade em $j \in \mathcal{M}^n$

$R_j$              Razão oferta/demanda da facilidade $j$, dependente de $x_j$ e $y_j$ <br>
$A_i$              Métrica de acessibilidade no nó $i$, dependente de $x_j$ e $y_j$ <br>
$Z$                Para linearização da função objetivo, tornando-a diferenciável

In [20]:
# Variáveis (Gurobi)

# y[j]: variável de decisão binária para abertura/operação de um centro em j ∈ M^n 
y = m.addVars(M_n, vtype=GRB.BINARY, name="y")

# x[j]: variável de decisão inteira para os novos suprimentos  j ∈ M^n
x = m.addVars(M_n, vtype=GRB.INTEGER, name="x")

# --- NOVAS VARIÁVEIS DE FUNÇÃO AUXILIARES ---
# R[j]: Razão oferta-demanda na localização j ∈ M
R = m.addVars(M, vtype=GRB.CONTINUOUS, name="R")

# A[i]: Score de acessibilidade no ponto de demanda i ∈ N
A = m.addVars(N, vtype=GRB.CONTINUOUS, name="A")

# Z: Equidade (acesso mínimo ponderado)
Z = m.addVar(vtype=GRB.CONTINUOUS, name="Z")
m.update()

Função Objetivo <br>
$\max \alpha \frac{1}{|\mathcal{N}|} \sum_{i \in \mathcal{N}} d_i A_i + (1 - \alpha)\, Z$

In [21]:
# --- FUNÇÃO OBJETIVO (MAXIMIZAR) ---
# 1. Eficiência (Acesso Médio Ponderado)
eficiencia_termo = gp.quicksum(d_i[i] * A[i] for i in N) / len(N)
# 2. Equidade (Acesso Mínimo)
equidade_termo   = Z

m.setObjective(alpha * eficiencia_termo + (1 - alpha) * equidade_termo, GRB.MAXIMIZE)

$R_j = 
    \begin{cases}
        \displaystyle \frac{x_j}{\sum_{i\in\mathcal{N}_j} w_{ij} d_i}, & j \in \mathcal{M}^n, \\[1em]
        \displaystyle \frac{\hat{x}_j}{\sum_{i\in\mathcal{N}_j} w_{ij} d_i}, & j \in \mathcal{M}^e,
    \end{cases}$

In [22]:
## RESTRIÇÕES

# 1. R1: Alocação de Suprimentos para a Razão R[j]
#    R[j] = Supply_j / Weighted_Demand_j
#    A restrição é reescrita como: R[j] * Weighted_Demand_j = Supply_j
for j in M:
    # Demanda Ponderada (Denominador) pré-calculado
    demanda_pond = DEMANDA_PONDERADA[j]
    
    if j in M_e:   # existente
        capac_total = x_hat[j]               # constante
    else:         # nova
        capac_total = x[j]                   # variável

    # Regra de Segurança: Evitar divisão por zero se a demanda ponderada for zero
    if demanda_pond > 0.0001:
        # Como demanda_pond é uma constante (parâmetro), a restrição é linear
        m.addConstr(R[j] * demanda_pond == capac_total, name=f"R1_Ratio_{j}")
    else:
        # Se a demanda ponderada for zero, R[e, j] não deve contribuir para a acessibilidade
        m.addConstr(R[j] == 0.0, name=f"R1_Ratio_Zero_Demanda_{j}")

$A_i = \sum_{j\in\mathcal{M}_i} w_{ij} R_j, \forall i \in \mathcal{N}$

In [23]:
# 2. R2: Cálculo da Acessibilidade A[i]
#    A[i] = Sum_{j in M_i} w_ij * R[j]
for i in N:           # i = "D_105", "D_333", etc.
    m.addConstr(
        A[i] == gp.quicksum(
            w_df.loc[i, j] * R[j]      # w_ij * R_j
            for j in M_i[i]            # facilidades cobertas por i
        ),
        name=f"R2_Acesso_{i}"
    )

$d_i A_i \ge Z, \quad \forall i \in \mathcal{N}$

In [24]:
# 3. R3: Restrição de Equidade (Linearização da parte MIN da FO)
#    Z <= Demanda_Ponderada Agregada * Score de Acessibilidade Agregada
# R6 Nova: Restrição de Equidade Mínima POR ESPECIALIDADE
for i in N:
    # Acesso Ponderado (d_i * A_i)
    # Como A[i] é uma variável do Gurobi, esta é a forma de obter o produto na restrição.
    acesso_ponderado_i = d_i[i] * A[i]
    
    # Z deve ser menor ou igual a CADA Acesso Ponderado (i)
    m.addConstr(acesso_ponderado_i >= Z, name=f"R3_Equidade_{i}")

$\underline{k}\, y_j \le x_j \le \bar{k}\, y_j, \forall j \in \mathcal{M}^n$

In [25]:
# 4. R4: Capacidade do Centro (Substitui R12)
# A soma dos especialistas alocados (x_j) deve estar entre [k_min, k_max] * y_j
# Se um centro é aberto (y_j=1), deve ter entre k_min e k_max FTEs.
for j in M_n:
    m.addConstr(k_min * y[j] <= x[j], name=f"R4_cap_min_{j}")
    m.addConstr(x[j] <= k_max * y[j], name=f"R4_cap_max_{j}")

$\sum_{j \in \mathcal{M}^n} y_j = P$

In [26]:
# 5. R5: Quantidade de Facilidades (Substitui antigo R8)
# ou (Max_Centros_Medicos)
m.addConstr(
    gp.quicksum(y[j] for j in M_n) == P_max,
    name="R5_Total_novas_facilidades"
)

# 6. R6: (Min_Centros_Medicos)
# Caso haja limites mínimos e máximos de Centros Médicos a serem abertos, adicionar essa restrição e 
# alterar '==' para '<=' na R5.
# m.addConstr(gp.quicksum(y[j] for j in M_n) >= P_min, name="R6_Min_Centros_Abertos")

<gurobi.Constr *Awaiting Model Update*>

$\sum_{j \in \mathcal{M}^n} x_j = Q$

In [27]:
# 7. R7: Orçamento em Suprimentos para Capacidade
m.addConstr(
    gp.quicksum(x[j] for j in M_n) == Q_max,
    name="R7_Total_suprimentos"
)

<gurobi.Constr *Awaiting Model Update*>

In [28]:
m.update()
print(f"\nModelo redefinido com: {m.numVars} variáveis, {m.numConstrs} restrições.")


Modelo redefinido com: 1165 variáveis, 1448 restrições.


In [29]:
# Configurações do ambiente para PERFORMANCE TUNING (a serem rodadas antes de m.optimize())
#m.setParam('TuneCriterion', 2) # 2 = Tempo (foco em velocidade). 1 = Gap (foco em qualidade)
## Define parâmetros para acelerar a solução
#m.setParam(GRB.Param.TimeLimit, 3600)  # Limite de 10 minutos
#m.setParam(GRB.Param.MIPGap, 0.001)    # Para quando o gap for <= 1%
#m.setParam(GRB.Param.Cuts, 2)       # Tenta melhorar o bound mais agressivamente
#m.setParam(GRB.Param.Heuristics, 0.5) # Aumenta a frequência de busca de soluções inteiras

# Execução da ferramenta de tuning
# O Gurobi irá testar combinações de parâmetros e buscar o melhor desempenho.
#m.tune() 

# Verifica se o tuning encontrou novos parâmetros
#if m.tuneResultCount > 0:
#    # Carrega o melhor conjunto de parâmetros
#    m.getTuneResult(0)
#    # Aplica o conjunto de parâmetros otimizado que o Gurobi encontrou
#    m.write('gurobi_tuned_parameters.prm')
#    m.read('gurobi_tuned_parameters.prm')
#    print("Parâmetros Otimizados Aplicados.")
#else:
#    print("O tuning não encontrou melhorias nos parâmetros.")

In [30]:
# Otimização
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license 2732058 - for non-commercial use only - registered to na___@usp.br
Optimize a model with 1448 rows, 1165 columns and 27582 nonzeros
Model fingerprint: 0x308e76af
Variable types: 601 continuous, 564 integer (282 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+02]
  Objective range  [9e-09, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 6e+02]
Presolve removed 882 rows and 601 columns
Presolve time: 0.02s
Presolved: 566 rows, 564 columns, 1692 nonzeros
Variable types: 0 continuous, 564 integer (282 binary)
Found heuristic solution: objective 0.0010908

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.06 seconds (0.01 work units)


In [31]:
# depois de m.optimize()
y_vals = {v.varName: v.x for v in m.getVars() if v.varName.startswith("y")}
x_vals = {v.varName: v.x for v in m.getVars() if v.varName.startswith("x")}
Z_val = m.getVarByName("Z").x

print("P abertas (soma y):", sum(y_vals.values()))
print("Q alocados (soma x):", sum(x_vals.values()))
print("Z (equidade):", Z_val)

P abertas (soma y): 3.0
Q alocados (soma x): 200.0
Z (equidade): 0.0


In [32]:
tol = 1e-6
mismatch = []
for j in M:  # M = lista de facilidades ("F_xxx")
    # tente localizar as variáveis R["F_xxx"] e x["F_xxx"] (ou x_hat para existentes)
    try:
        Rj = R[j]  # ou o nome que você usou
    except:
        Rj = None
    # recupere demanda_pond de dict DEMANDA_PONDERADA[j]
    denom = DEMANDA_PONDERADA.get(j, 0.0)
    supply = x_vals.get(f"x[{j}]", x_hat.get(j, 0.0))  # adapte os nomes
    if denom > 1e-8 and Rj is not None:
        lhs = Rj.x * denom
        if abs(lhs - supply) > tol:
            mismatch.append((j, lhs, supply))
    else:
        # se denom==0 então espera-se Rj==0
        if Rj is not None and abs(Rj.x) > tol:
            mismatch.append((j, "R_nonzero_but_denom0", Rj.x))
print("R mismatches:", mismatch[:10])

R mismatches: []


In [33]:
Ai_mismatch = []
for i in N:  # N = lista "D_xxx"
    Ai_var = A[i]  # ajuste conforme nomes reais
    computed = sum(w_df.loc[i, j] * R[j].x for j in M_i[i])
    if abs(Ai_var.x - computed) > 1e-6:
        Ai_mismatch.append((i, Ai_var.x, computed))
print("A mismatches:", Ai_mismatch[:10])

# check equity
violations = [i for i in N if d_i[i]*A[i].x + 1e-9 < Z_val]
print("violações de equidade (deveria ser none):", violations[:10])

A mismatches: []
violações de equidade (deveria ser none): []


In [34]:
fixed = []
for v in m.getVars():
    if abs(v.lb - v.ub) < 1e-12:
        fixed.append((v.varName, v.lb))
print("variáveis fixadas:", len(fixed), fixed[:10])

variáveis fixadas: 0 []


In [35]:
if m.status == GRB.OPTIMAL:

    print("\n================= RELATÓRIO DO MODELO =================\n")

    # OBJETIVO
    print(f"Valor ótimo da Função Objetivo: {m.objVal:.6f}")

    # Z - equidade
    Z_val = m.getVarByName("Z").x
    print(f"Valor de Z (Equidade): {Z_val:.6f}\n")

    # ----------------------------------------------------
    # 1. ANALISAR Y_j (facilidades abertas)
    # ----------------------------------------------------
    print(">>> FACILIDADES ABERTAS (y_j = 1):")
    abertas = []
    for j in M_n:
        yj = y[j].x
        if yj > 0.5:
            abertas.append(j)
    print(f"Total abertas: {len(abertas)}")
    print("Lista:", abertas, "\n")

    # ----------------------------------------------------
    # 2. ANALISAR X_j (novos suprimentos)
    # ----------------------------------------------------
    print(">>> NOVOS SUPRIMENTOS (x_j > 0):")
    novos = []
    total_x = 0
    for j in M_n:
        xj = x[j].x
        if xj > 1e-8:
            novos.append((j, xj))
            total_x += xj

    print(f"Total de novos suprimentos alocados: {total_x}")
    for item in novos:
        print(f" - {item[0]}: {item[1]:.4f}")
    print()

    # ----------------------------------------------------
    # 3. ANALISAR R_j
    # ----------------------------------------------------
    print(">>> VALORES DE R_j (razão oferta/demanda):")
    R_vals = {}
    for j in M:
        var = R[j]
        if var is not None:
            R_vals[j] = var.x

    print("Exemplo dos 10 primeiros R_j:")
    for j in list(R_vals.keys())[:10]:
        print(f" - {j}: {R_vals[j]:.6f}")
    print()

    # ----------------------------------------------------
    # 4. ANALISAR ACESSIBILIDADE A_i
    # ----------------------------------------------------
    print(">>> ACESSIBILIDADE A_i E VERIFICAÇÃO DE Z:")
    A_vals = {}
    acessos = []
    zeros = []

    for i in N:
        Ai = A[i].x
        A_vals[i] = Ai
        prod = d_i[i] * Ai
        acessos.append((i, prod))

        if prod <= 1e-8:
            zeros.append((i, d_i[i], Ai))

    pior = min(acessos, key=lambda x: x[1])
    melhor = max(acessos, key=lambda x: x[1])

    print(f"Pior ponto em d_i*A_i: {pior[0]} = {pior[1]:.6f}")
    print(f"Melhor ponto em d_i*A_i: {melhor[0]} = {melhor[1]:.6f}\n")

    print("Pontos que geram Z = 0:")
    for z in zeros[:20]:
        print(f" - i={z[0]}, d_i={z[1]}, A_i={z[2]}")
    print()

    # ----------------------------------------------------
    # 5. Verificação das restrições R[j] * DEMANDA -> supply
    # ----------------------------------------------------
    print(">>> VERIFICAÇÃO DAS RESTRIÇÕES R_j * DEMANDA_POND = SUPPLY_j")
    inconsistencias = []
    tol = 1e-6
    for j in M:
        denom = DEMANDA_PONDERADA.get(j, 0.0)
        Rj = R[j].x
        if j in M_e:
            supply = x_hat.get(j, 0.0)    # existente -> constante
        else:
            supply = x[j].x               # nova -> variável
        if denom > 1e-8:
            if abs(Rj * denom - supply) > tol:
                inconsistencias.append((j, Rj * denom, supply))
        else:
            if abs(Rj) > tol:
                inconsistencias.append((j, "denom_zero_but_R_nonzero", Rj))
    print("Inconsistências:", inconsistencias[:15])

    print("=============== FIM DO RELATÓRIO ===============")

else:
    print("Modelo não ótimo. Status:", m.status)


================= RELATÓRIO DO MODELO =================

Valor ótimo da Função Objetivo: 0.001091
Valor de Z (Equidade): 0.000000

>>> FACILIDADES ABERTAS (y_j = 1):
Total abertas: 3
Lista: ['D_280370805_.', 'D_280480511_2804805001', 'D_280480511_.'] 

>>> NOVOS SUPRIMENTOS (x_j > 0):
Total de novos suprimentos alocados: 200.0
 - D_280370805_.: 100.0000
 - D_280480511_2804805001: 50.0000
 - D_280480511_.: 50.0000

>>> VALORES DE R_j (razão oferta/demanda):
Exemplo dos 10 primeiros R_j:
 - F_2800300002232: 13.076860
 - F_2800300002275: 0.500802
 - F_2800300002283: 1.104625
 - F_2800300002372: 0.092696
 - F_2800300002534: 0.496330
 - F_2800300002585: 0.751538
 - F_2805402420120: 5.126931
 - F_2800202421488: 2.367708
 - F_2803502421518: 0.663760
 - F_2804502421542: 2.419519

>>> ACESSIBILIDADE A_i E VERIFICAÇÃO DE Z:
Pior ponto em d_i*A_i: D_280340105_. = 0.000000
Melhor ponto em d_i*A_i: D_280350005_. = 98.379018

Pontos que geram Z = 0:
 - i=D_280340105_., d_i=10.609463927191442, A_i=0

In [36]:
####### PARA MAPEAMENTO DA ACESSIBILIDADE NO QGIS
A_depois = {i: A[i].x for i in N}
for i in list(N)[:10]:
    print(i, A_depois[i])

D_280190005_. 0.27055493813239145
D_280480517_2804805002 3.483046067433348
D_280530705_. 1.8694394318338674
D_280430005_. 2.1211360834472504
D_280480505_. 3.1508039978439086
D_280067005_2800670001 0.2394530162918322
D_280067005_2800670003 0.23660844938037964
D_280067005_2800670002 0.2282320363631143
D_280067005_2800670005 0.24848595832290224
D_280067005_2800670004 0.19989694122885562


In [37]:
# 1. calcular R_j antes
R_antes = {}
for j in M:
    denom = DEMANDA_PONDERADA.get(j, 0.0)
    
    if j in M_e:   # facilidade existente
        supply_j = x_hat[j]   # seu dict de capacidades existentes
    else:
        supply_j = 0.0        # não existe antes
    
    if denom > 1e-8:
        R_antes[j] = supply_j / denom
    else:
        R_antes[j] = 0.0

# 2. calcular A_i antes
A_antes = {}
for i in N:
    soma = 0.0
    for j in M_i[i]:                 # facilidades que cobrem o i
        soma += w_df.loc[i, j] * R_antes[j]
    A_antes[i] = soma

In [38]:
df_acess = pd.DataFrame({
    "ID_modelo": list(N),
    "A_antes": [A_antes[i] for i in N],
    "A_depois": [A_depois[i] for i in N],
})
df_acess["A_delta"] = df_acess["A_depois"] - df_acess["A_antes"]

In [39]:
def id_para_qgis(id_modelo):
    return id_modelo.replace("D_", "")

df_acess["ID_qgis"] = df_acess["ID_modelo"].apply(id_para_qgis)

In [40]:
df_acess.to_csv("acessibilidade_qgis.csv", index=False)

In [55]:
df_ids = pd.DataFrame({"ID_divisao": abertas})
df_ids["ID_qgis"] = df_ids["ID_divisao"].apply(id_para_qgis)
df_ids.to_csv("facilidades_selecionadas.csv", index=False)
df_ids

,ID_divisao,ID_qgis
0,D_280370805_.,280370805_.
1,D_280480511_2804805001,280480511_2804805001
2,D_280480511_.,280480511_.
